In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Dense, Dropout, LSTM, Input, Bidirectional
from keras.models import Sequential
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Load the dataset
data = pd.read_csv("DataSet.csv")

# Feature Engineering: Create moving averages
data['MA_10'] = data['Close'].rolling(window=10).mean()
data['MA_50'] = data['Close'].rolling(window=50).mean()
data['MA_100'] = data['Close'].rolling(window=100).mean()
data.dropna(inplace=True)  # Drop NA values

# Normalize the features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data[['Close', 'MA_10', 'MA_50', 'MA_100']])

# Prepare the training and test datasets
train_size = int(len(scaled_data) * 0.80)
data_train = scaled_data[:train_size]
data_test = scaled_data[train_size:]

# Create the x and y datasets
def create_dataset(dataset, time_step=1):
    x, y = [], []
    for i in range(len(dataset) - time_step - 1):
        x.append(dataset[i:(i + time_step), :])
        y.append(dataset[i + time_step, 0])  # Predicting Close price
    return np.array(x), np.array(y)

time_step = 100
x_train, y_train = create_dataset(data_train, time_step)
x_test, y_test = create_dataset(data_test, time_step)

# Build the LSTM model
model = Sequential()
model.add(Input(shape=(x_train.shape[1], x_train.shape[2])))
model.add(Bidirectional(LSTM(units=50, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=50, return_sequences=False)))
model.add(Dropout(0.2))
model.add(Dense(units=1))  # Output layer

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Fit the model
model.fit(x_train, y_train, epochs=100, batch_size=32, validation_split=0.2, verbose=1)

# Make predictions
y_train_pred = model.predict(x_train)
y_test_pred = model.predict(x_test)

# Inverse scaling to get actual prices
y_train_pred = scaler.inverse_transform(np.concatenate((y_train_pred, np.zeros((y_train_pred.shape[0], 3))), axis=1))[:, 0]
y_test_pred = scaler.inverse_transform(np.concatenate((y_test_pred, np.zeros((y_test_pred.shape[0], 3))), axis=1))[:, 0]
y_train_actual = scaler.inverse_transform(np.concatenate((y_train.reshape(-1, 1), np.zeros((y_train.shape[0], 3))), axis=1))[:, 0]
y_test_actual = scaler.inverse_transform(np.concatenate((y_test.reshape(-1, 1), np.zeros((y_test.shape[0], 3))), axis=1))[:, 0]

# Calculate evaluation metrics
mae = mean_absolute_error(y_test_actual, y_test_pred)
mse = mean_squared_error(y_test_actual, y_test_pred)
rmse = np.sqrt(mse)
r_squared = r2_score(y_test_actual, y_test_pred)

# Print the evaluation results
print(f'Mean Absolute Error (MAE): {mae}')
print(f'Mean Squared Error (MSE): {mse}')
print(f'Root Mean Squared Error (RMSE): {rmse}')
print(f'R-squared (R²): {r_squared}')

# Plot the results
plt.figure(figsize=(12, 6))
plt.plot(y_test_actual, label='Actual Price', color='g')
plt.plot(y_test_pred, label='Predicted Price', color='r')
plt.title('Stock Price Prediction')
plt.xlabel('Days')
plt.ylabel('Price')
plt.legend()
plt.show()